In [1]:
## LIBRARIES IMPORT FOR TROPIDASH APP ##
 
from eccodes import *
from datetime import datetime, timedelta

import requests
import pandas as pd
import numpy as np
import ipywidgets as widgets

from IPython.display import Javascript, display, clear_output

import warnings
warnings.filterwarnings("ignore")

import utils_tracks as utr
from utils_atm import * 
from utils_impacts import *
from utils_section4 import *
import utils_temporal as utt

In [2]:
## CREATE FOLDERS FOR DATA STORAGE ##

# Atmospheric data folder
os.makedirs('data/atm', exist_ok=True)

# Tracks data folder
os.makedirs('data/tracks', exist_ok=True)

In [3]:
## WIDGET TO IMPORT THE TITLE IMAGE ##

file = open("data/layout/title.jpg", "rb")
image = file.read()
title_img = widgets.Image(
    value=image,
    format='jpg',
)

In [4]:
## WIDGET TO DEFINE THE STARTING DATE OF THE FORECAST ##
## This sets the date of which we want to download the forecast ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

In [5]:
## DOWNLOAD THE FORECAST TRACKS ##
# The forecast data is saved as bufr file called tc_test_track_data.bufr    
start_date = utr.download_tracks_forecast(start_date_forecast.value)

# Boolean to remember if the forecast is from yesterday for eventual warnings in the app
if start_date != start_date_forecast.value:
    yesterday_forecast = True
else:
    yesterday_forecast = False

## DOWNLOAD THE OBSERVED TRACKS DATA FROM IBTrACS ##
# The observed data is saved as a csv file called ibtracs.ACTIVE.list.v04r00.csv
url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/tracks/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

In [6]:
## LOAD THE FORECAST DATA OF THE TRACKS IN A DATAFRAME ##
df_storms_forecast = utr.create_storms_df(start_date)

## LOAD THE OBSERVED TRACKS IN A DATAFRAME ## 
df_storms_observed = pd.read_csv('data/tracks/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

## CREATE THE LIST OF ACTIVE CYCLONES IN THE FORECAST ##
cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]

In [7]:
## WIDGET TO DISPLAY A MESSAGE REGARDING THE ACTIVE CYCLONES ##

if df_storms_forecast.empty:
    str1 = f"<center>There are <font color='orange'><b>NO active cyclones</b></font> in the forecast of <b>{start_date_forecast.value.strftime('%d %b %Y')}.</b></center>"
    str2 = "<center>\n<font color='red'>Please select another date for the forecast.</font></center>"
    message = str1 + str2
else:
    n_cyclones = len(df_storms_forecast.stormIdentifier.unique())
    if n_cyclones == 1:
        str0 = f"<center>There is <font color='orange'><b>1 active cyclone </b></font>"
    else:
        str0 = f"<center>There are <font color='orange'><b>{n_cyclones} active cyclones </b></font>"
    str1 = f"in the forecast of <b>{start_date_forecast.value.strftime('%d %b %Y')}.</b></center>"
    str2 = "\n<center>Please click the dropdown menu to check the active storms in the forecast.</center>"
    str3 = "\n<center>The list may take a few seconds to update.</center>"
    message = str0 + str1 + str2 + str3

message_cyclone = widgets.HTML(
    value=message,
)

def update_message_cyclone(_):
    start_date = utr.download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = utr.create_storms_df(start_date)
    n_cyclones = len(df_storms_forecast.stormIdentifier.unique())
    if df_storms_forecast.empty:
        str1 = f"<center>There are <font color='orange'><b>NO active cyclones</b></font> in the forecast of <b>{start_date_forecast.value.strftime('%d %b %Y')}.</b></center>"
        str2 = "<center>\n<font color='red'>Please select another date for the forecast.</font></center>"
        message = str1 + str2
    else:
        n_cyclones = len(df_storms_forecast.stormIdentifier.unique())
        if n_cyclones == 1:
            str0 = f"<center>There is <font color='orange'><b>1 active cyclone </b></font>"
        else:
            str0 = f"<center>There are <font color='orange'><b>{n_cyclones} active cyclones </b></font>"
        str1 = f"in the forecast of <b>{start_date_forecast.value.strftime('%d %b %Y')}.</b></center>"
        str2 = "\n<center>Please click the dropdown menu to check the active storms in the forecast.</center>"
        str3 = "\n<center>The list may take a few seconds to update.</center>"
        message = str0 + str1 + str2 + str3
    message_cyclone.value = message

# Update the message when the date is changed
start_date_forecast.observe(update_message_cyclone, 'value')

In [8]:
## WIDGET TO SHOW THE LIST OF ACTIVE CYCLONES AND SELECT THE ONE OF WHICH WE WANT TO PLOT THE DATA ##

cyclone = widgets.Dropdown(
    options = active_cyclones,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'
cyclone.style.font_weight = 'bold'

# Update storms list when downloading new tracks data
def update_storms_list(_):
    start_date = utr.download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = utr.create_storms_df(start_date)
    cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
    active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]
    cyclone.options = active_cyclones

start_date_forecast.observe(update_storms_list, names='value')

In [9]:
## WIDGET TO SELECT WHICH ENSEMBLE MEMBERS OF THE FORECAST TRACK PLOT ##

members = df_storms_forecast.ensembleMemberNumber.unique().tolist()
ens_members = widgets.SelectMultiple(
    options=members,
    value=[],
    description="Ensemble members to plot:",
    disable=False
)

ens_members.style.description_width = '168px'
ens_members.style.font_weight = 'bold'

# Update ensemble members list when new track data are available
def update_ensembles_list(_):
    start_date = utr.download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = utr.create_storms_df(start_date)
    ens_members.options = df_storms_forecast.ensembleMemberNumber.unique().tolist()
    
cyclone.observe(update_ensembles_list, names='value')
    
# Print widget
message_ens = widgets.HTML(
    value="Multiple values can be selected with <b>shift</b> and/or <b>ctrl</b> (or <b>command</b>)",
)

In [10]:
## UTILITIES FOR SECTION 2 (ATMOSPHERIC VARIABLES) ##
## VARIABLES DEFINITION ##
variables = ['msl', 'skt', 'tp', 'wind', '10fgg25']

In [11]:
## UTILITIES FOR SECTION 3 (IMPACTS) ##
## WIDGETS FOR RETURN PERIOD SELECTION ##

vec_rp_coh = ["5yr", "10yr", "50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_coh = widgets.Dropdown(options = vec_rp_coh, description = 'Return Period Coastal Hazard')
widget_sel_rp_coh.style.description_width = "180px"

vec_rp_cyh = ["50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_cyh = widgets.Dropdown(options = vec_rp_cyh, description = 'Return Period Cyclone Hazard')
widget_sel_rp_cyh.style.description_width = "180px"

In [13]:
## UPDATE BUTTON TO PLOT NEW TRACKS AND DATA DEPENDING ON THE CYCLONE SELECTED ##

update_button = widgets.Button(
    description = 'Update Forecast',
    icon = "check",
    style=dict(
        button_color='blue',
        font_weight='bold',
        text_color='lightgreen',
        text_decoration='underline',
))

update_output = widgets.Output()

## Function to create a html widget that insert a title for each section of the app ##

def create_section_title(title):
    title = widgets.HTML(
        value=f"<h1>{title}</h1>",
        placeholder='',
        description='',
    )
    return title

## Plot new data when clicking the Update Forecast Button ##
def on_button_clicked(b):
    with update_output:
        ## Display Section 1 Title
        display(create_section_title("Section 1 - Tropical Cyclone Tracks and Characteristics"), clear=True)
        ## Output of initial lat-lon, average track and initial-final time step of the cyclone for the next sections ##
        code, name = cyclone.value.split('-')
        display(widgets.HTML(value=f"Creating the dataset for tropical cyclone <b>{name}</b>")) # clear previous cyclones plots
        df_storms_forecast = utr.create_storms_df(start_date_forecast.value)
        df_storms_observed = pd.read_csv('data/tracks/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
        df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
        df_f.reset_index(drop=True, inplace=True)
        df_o = df_storms_observed[df_storms_observed.NAME.squeeze() == name]
        df_o.reset_index(drop=True, inplace=True)
        # Initial and final lat-lon
        initial_lat_lon = (df_f.latitude.iloc[0], df_f.longitude.iloc[0])
        last_position_index = -1
        while np.isnan(df_f.latitude.iloc[last_position_index]):
            last_position_index = last_position_index - 1
            if last_position_index == len(df_f.latitude):
                print('Error: no final forecasted position')
                break
        final_lat_lon = (df_f.latitude.iloc[last_position_index], df_f.longitude.iloc[last_position_index])
        coord = ((initial_lat_lon[0] + final_lat_lon[0])/2, (initial_lat_lon[1] + final_lat_lon[1])/2)

        # Average track
        avg_info = utr.mean_forecast_track(df_f)
        locations_avg = avg_info[0]
        # Initial and final time step
        df_f["date"] = datetime(df_f.year[0], df_f.month[0], df_f.day[0], df_f.hour[0]) + timedelta(hours=1) * df_f.timePeriod
        # If we select a forecast date in the past the cyclone might not be present in the observations data because we can access only the latest one
        if len(df_o) == 0:
            display(widgets.HTML(value=f"Cyclone <b>{name}</b> is not present in the latest observations data of IBTrACS"))
        final_timestep = df_f.date.iloc[-1].to_pydatetime()
        display(widgets.HTML(value="Dataset prepared!"))

        ## Plots of the tracks ## 
        display(widgets.HTML(value=f"Creating the plots for the tropical cyclone tracks considering ensemble members: <b>{ens_members.value}</b>"))
        tc_track_map, cyclonelayers = utr.plot_cyclone_tracks_ipyleaflet(ens_members.value, df_f, df_o)
        display(tc_track_map)

        ## Section 2 ##
        ## Plot of atmopsherical variables ##
        display(create_section_title("Section 2 - Atmospheric Variables"))
        
        # Compute the steps to be downloaded
        cyclone_days = len(pd.date_range(start = start_date_forecast.value.strftime("%Y%m%d"),
                                            end = final_timestep.strftime("%Y%m%d")))
        windprobsteps = [f"{12 * i}-{ 12 * i + 24}" for i in range(2*cyclone_days-1)]
        windprobsteps.insert(0, "0-24") #add to be able to plot when 12h forecast is selected
        stepsdict = {
                            "base": list(np.arange(12, 240, 12)[0:2*cyclone_days]),
                            "10fgg25": windprobsteps
                        }
        # Widget for forecast step selection
        vec_forecast = [f"{x}h from selected date" for x in stepsdict["base"]]
        widget_sel_forecast = widgets.Dropdown(options = vec_forecast, description = 'Forecast')

        display(widgets.HTML(value = "Downloading the forecasted atmospheric variables"))
        fnames = dwnl_atmdata(variables, stepsdict, start_date_forecast.value, pr = False);
        vardict = load_atmdata(variables, fnames)
        
        display(widgets.HTML(value = "Plotting the forecasted atmospheric variables"))
        display(widgets.interactive(plot_atmdata_step, 
                                    step = widget_sel_forecast, 
                                    vardict = widgets.fixed(vardict),
                                    coord =  widgets.fixed(coord), 
                                    stepsdict =  widgets.fixed(stepsdict)))
        ## Section 3 ##
        ## Plots of impact variables ## 
        display(create_section_title("Section 3 - Impact Variables"))
        
        display(widgets.HTML(value = "Plotting the impact layers"))
        display(widgets.interactive(impacts_plot, rp_coh = widget_sel_rp_coh, rp_cyh = widget_sel_rp_cyh, coord = widgets.fixed(coord)))

        ## Section 4 ##
        display(create_section_title("Section 4 - Joint visualization: cyclones, atmospheric, impacts"))
        display(widgets.HTML(value = "The plot contains a selection of cyclone, atmospheric and impacts variables"))
        
        display(widgets.HTML(value = "Plotting the joint visualization"))
        display(widgets.interactive(
                                    plot_section4,
                                    step = widget_sel_forecast,
                                    rp_coh = widget_sel_rp_coh,
                                    rp_cyh = widget_sel_rp_cyh,
                                    vardict = widgets.fixed(vardict),
                                    coord =  widgets.fixed(coord),
                                    stepsdict =  widgets.fixed(stepsdict),
                                    cyclonelayers = widgets.fixed(cyclonelayers)
                                    ))
        
        ## Section 5 ##            
        ## Map of temporal evolution ##
        display(create_section_title("Section 5 - Temporal Evolution"))
        # Print widget
        display(widgets.HTML(
                value = "Move the pointer to get the temporal evolution on that position. Scroll to see the other variables.",
                ))

        maps5 = utt.create_maps_s5(initial_lat_lon, locations_avg, fnames)
        display(maps5)

update_button.on_click(on_button_clicked)

In [14]:
## WIDGET FOR INFOMATION BUTTON ##

info_button = widgets.Button(
    description = 'TropiDash Info',
    icon = "info",
    style=dict(
        button_color='orange',
        font_weight='bold',
        text_color='black',
        text_decoration='underline',
        border='solid black 1px',
))

notify_output = widgets.Output()
update_info = widgets.Output()
display(notify_output)

@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))

def clickme(b):
    info_message = "Information regarding the Dashboard are displayed below all sections graphs."
    popup(info_message)
    with update_info:
        display(widgets.HTML(value="<center><h1>Tropi<font color='orange'>Dash</font> Information</h1></center>"), clear=True)
        display(widgets.HTML(value="TropiDash is an interactive dashboard to visualize and consult tropical cyclone data. It is composed of 5 sections:"
                             "<ol type='1'>" "<li>Tropical Cyclone Tracks and Characteristics</li>" 
                             "<li>Atmospheric Variables</li>" 
                             "<li>Impact Variables</li>" 
                             "<li>Multi-Layer Map combining data from all previous sections</li>" 
                             "<li>Point-Wise Temporal Evolution of Atmospheric Variables</li> </ol>"))
        display(widgets.HTML(value="All documents regarding the dashboard are available at the following <a href='https://github.com/ECMWFCode4Earth/TropiDash' target='_blank'><font color='blue'><u>github repository</u></font></a>."))
        display(widgets.HTML(value="You can find more information about each section and the data sources in the <a href='https://github.com/ECMWFCode4Earth/TropiDash/blob/main/TropiDash_documentation.md' target='_blank'><font color='blue'><u>dashboard documentation</u></font></a>."))
        display(widgets.HTML(value="For specific tutorial examples please refer to the folder <a href='https://github.com/ECMWFCode4Earth/TropiDash/tree/main/tutorials' target='_blank'><font color='blue'><u>tutorials</u></font></a>."))
        display(widgets.HTML(value="To use the dashboard, select the forecast date, the active cyclone, the numbers of ensembles to plot and click the <img src='data/layout/update_button.jpg' width='70'> button."
                             " The dashboard will automatically update the plots and the data displayed in the sections."))
        display(widgets.HTML(value="If you want to install the dashboard locally, please find the required packages to run the notebook in the environment file <a href='https://github.com/ECMWFCode4Earth/TropiDash/blob/main/environment.yml' target='_blank'><font color='blue'><u>requirements.txt</u></font></a>."))
        display(widgets.HTML(value="For any questions or feedback please contact the dashboard developers:"
                             "<ul> <li> Filippo Dainelli <a href='mailto:filippo.dainelli@polimi.it'><font color='blue'><u>send email</u></font></a></li>"
                             "<li> Paolo Colombo <a href='mailto:paolo1.colombo@polimi.it'><font color='blue'><u>send email</u></font></a></li>"
                             "<li> Laura Paredes i Fortuny <a href='mailto:paredes_laufor@externos.gva.es <paredes_laufor@externos.gva.es>'><font color='blue'><u>send email</u></font></a></li></ul>"))
        display(widgets.HTML(value="<center><h3>Thank you for using Tropi<font color='orange'>Dash</font>!</h3></center>"))
info_button.on_click(clickme)

Output()

In [15]:
## GROUP THE THREE PRINCIPAL WIDGET IN A VERTICAL BOX STRUCTURE ##
if yesterday_forecast:
    forecast_day_message = widgets.HTML(value=f"<center><b><font color='red'>WARNING:</font></b> The forecast from <b>{start_date_forecast.value.strftime('%d %b %Y')}</b> is not available yet.</center>" 
                                        f"<center><b>{start_date.strftime('%d %b %Y')}</b> will be used instead, please select this date to process the dataset accordingly.</center>")
    warning_box = widgets.Box([forecast_day_message],
                    layout=widgets.Layout(
                        display='flex',
                        border='3px solid red',
                        align_items='center',
                        width='100%'
                    ))
    group_box = widgets.Box([start_date_forecast, message_cyclone, cyclone, message_ens, ens_members],
                  layout=widgets.Layout(
                      display='flex',
                      flex_flow='column',
                      border='3px solid orange',
                      align_items='center',
                      width='100%'
                  ))
    selection_box = widgets.Box([warning_box, group_box],
                    layout=widgets.Layout(
                        display='flex',
                        flex_flow='column',
                        align_items='center',
                        width='100%'
                    ))
else:
    selection_box = widgets.Box([start_date_forecast, message_cyclone, cyclone, message_ens, ens_members],
                    layout=widgets.Layout(
                        display='flex',
                        flex_flow='column',
                        border='3px solid orange',
                        align_items='center',
                        width='100%'
                    ))

## GROUP THE THREE PRINCIPAL WIDGET AND THE UPDATE BUTTON TOGETHER
widgets_box = widgets.Box([selection_box, update_button],
                          layout=widgets.Layout(
                              display='flex',
                              flex_flow='column',
                              align_items='center',
                              # align_content='space-between',
                              # width='100%',
                              # height='100%'
                          ))

## GROUP THE TITLE IMAGE WITH THE INFORMATION BUTTON ##
info_box = widgets.Box([title_img, info_button],
                        layout=widgets.Layout(
                            display='flex',
                            flex_flow='column',
                            align_items='center',
                        ))


## GROUP THE TITLE IMAGE WITH THE GENERAL WIDGETS BOX ##
title_box = widgets.Box([info_box, widgets_box],
                        layout=widgets.Layout(
                            display='flex',
                            flex_flow='row',
                            align_items='center'
                        ))

## GROUP THE WIDGETS, THE GRAPHS AND THE INFORMATION DISPLAY ## 
display_box = widgets.Box([title_box, update_output, update_info],
                          layout=widgets.Layout(
                                display='flex',
                                flex_flow='column',
                                align_items='center',
                          ))

display(display_box)

Box(children=(Box(children=(Box(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\xcd\x…